In [1]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit, KFold
import random
import numpy as np

# import data

In [2]:
files = glob.glob('/code/media/QNAP/People/GongyuZhang/ROP/data/raw/image/*')
filename_fromglob=[]
filename_fromglob_full=[]
for f in files:
    filename_fromglob.append(os.path.split(f)[1][:36])
    filename_fromglob_full.append(os.path.split(f)[1])
print(filename_fromglob_full[:5])

['03255041-b3a3-4fc2-b449-b4cae007716e.7.png', '0597ab91-77e4-47bd-8e09-a3eb342a6470.31.png', '08ab5d5c-e25e-4701-a5df-c50d512f3fde.23.png', '0bfef26d-7239-4453-a6df-34e75cb69125.2.png', '14a711a2-302e-49f4-b5ca-e4bc0d716839.12.png']


In [3]:
df_train_original = pd.read_csv('/code/media/QNAP/People/GongyuZhang/ROP/data/raw/Grading/train_18_April.csv')
df_test_original = pd.read_csv('/code/media/QNAP/People/GongyuZhang/ROP/data/raw/Grading/test_18_April.csv')
df_entire = pd.concat([df_train_original, df_test_original])
# add missing grading
df_entire.loc[len(df_entire)]=[98, '115b4818-6213-4bfb-92a7-07f7d7303f53.7.png',np.NaN,np.NaN,np.NaN,'normal','normal','normal']
df_entire.loc[df_entire['id'] == 98]

,id,Filepath,CH,JT,KP,SB,GA,group
414,98,115b4818-6213-4bfb-92a7-07f7d7303f53.1.png,NaN,NaN,normal,normal,normal,normal
415,98,115b4818-6213-4bfb-92a7-07f7d7303f53.14.png,normal,normal,NaN,NaN,normal,normal
416,98,115b4818-6213-4bfb-92a7-07f7d7303f53.2.png,normal,normal,NaN,NaN,normal,normal
417,98,115b4818-6213-4bfb-92a7-07f7d7303f53.39.png,normal,normal,NaN,NaN,normal,normal
7413,98,115b4818-6213-4bfb-92a7-07f7d7303f53.7.png,NaN,NaN,NaN,normal,normal,normal


# Creat test set (50normal + 50ungra + 50referable)

In [4]:
random.seed(88)
df_test_cal = df_entire.copy()

df_test_refer = df_test_cal.loc[(df_test_cal['group'] == 'plus') | (df_test_cal['group'] == 'preplus')].drop_duplicates(subset=['id']).sample(n=50)
df_test_cal = df_test_cal[~df_test_cal.id.isin(df_test_refer.id)]

df_test_normal = df_test_cal.loc[(df_test_cal['group'] == 'normal')].drop_duplicates(subset=['id']).sample(n=50)
df_test_cal = df_test_cal[~df_test_cal.id.isin(df_test_normal.id)]

df_test_ungradable = df_test_cal.loc[(df_test_cal['group'] == 'ungradable')].drop_duplicates(subset=['id']).sample(n=50)

print(set(df_test_ungradable.id) & set(df_test_normal.id))
print(set(df_test_normal.id) & set(df_test_refer.id))
print(set(df_test_ungradable.id) & set(df_test_refer.id))

print(len(df_test_ungradable))
print(len(df_test_normal))
print(len(df_test_refer))
id_testset = df_test_refer.id + df_test_ungradable.id + df_test_normal.id
df_test = pd.concat([df_test_refer,df_test_normal,df_test_ungradable])
df_test_dropped = df_entire[(~df_entire.Filepath.isin(df_test.Filepath)) & (df_entire.id.isin(df_test.id))]
df_train_val = df_entire[~df_entire["id"].isin(df_test.id)]
print(len(set(id_testset)))
print(len(df_train_val) + len(df_test) + len(df_test_dropped))

set()
set()
set()
50
50
50
148
7414


# Creat 5 fold for train-val set

In [5]:
folds=[]
ids = df_train_val.id.unique().tolist()
id2split = ids

for f in range(5):
    fold = random.sample(id2split, int(len(ids)/5))
    fold.sort()
    folds.append(fold)
    id2split = [x for x in id2split if x not in fold]

print(set(folds[0])&set(folds[1])&set(folds[2])&set(folds[3])&set(folds[4]))
print()

set()



# Check numbers in 5 fold train-val

In [6]:
def show_class_stat(df):
    print('\nnum={}'.format(len(df)))
    print(df.loc[:,"group"].value_counts(normalize=False))
    print("{}".format(df.loc[:,"group"].value_counts(normalize=True)))

    
df_test['split'] = 'test'
df_test_dropped['split'] = 'test_dropped'

df_f1 = df_entire.loc[df_entire.id.isin(folds[0]),]
df_f1['split'] = 'f1'
df_f2 = df_entire.loc[df_entire.id.isin(folds[1]),]
df_f2['split'] = 'f2'
df_f3 = df_entire.loc[df_entire.id.isin(folds[2]),]
df_f3['split'] = 'f3'
df_f4 = df_entire.loc[df_entire.id.isin(folds[3]),]
df_f4['split'] = 'f4'
df_f5 = df_entire.loc[df_entire.id.isin(folds[4]),]
df_f5['split'] = 'f5'

print('----------------')
print('test set')
show_class_stat(df_test)
print('----------------')
print('five folds')

for df in [df_f1,df_f1,df_f1,df_f1,df_f1]:
    show_class_stat(df)

----------------
test set

num=150
ungradable    50
normal        50
preplus       36
plus          14
Name: group, dtype: int64
ungradable    0.333333
normal        0.333333
preplus       0.240000
plus          0.093333
Name: group, dtype: float64
----------------
five folds

num=1358
normal        1235
ungradable      64
preplus         37
plus            22
Name: group, dtype: int64
normal        0.909426
ungradable    0.047128
preplus       0.027246
plus          0.016200
Name: group, dtype: float64

num=1358
normal        1235
ungradable      64
preplus         37
plus            22
Name: group, dtype: int64
normal        0.909426
ungradable    0.047128
preplus       0.027246
plus          0.016200
Name: group, dtype: float64

num=1358
normal        1235
ungradable      64
preplus         37
plus            22
Name: group, dtype: int64
normal        0.909426
ungradable    0.047128
preplus       0.027246
plus          0.016200
Name: group, dtype: float64

num=1358
normal        123

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [7]:
df_splited = pd.concat([df_test,df_test_dropped,df_f1,df_f2,df_f3,df_f4,df_f5])
df_splited.reset_index(drop=True, inplace=True)

In [8]:
# check missing id
[x for x in range(max(set(df_splited.id.unique()))) if x not in df_splited.id.unique()]

[]

In [9]:
df_splited.to_csv('splitted.csv')

In [10]:
df = pd.read_csv('splitted.csv')

In [11]:
print(len(set(df[(df.split == "test") & ((df.group == "preplus") | (df.group == "plus"))].id)))
print(len(set(df[(df.split == "test") & ((df.group == "normal"))].id)))
print(len(set(df[(df.split == "test") & ((df.group == "ungradable"))].id)))

50
50
50
